In [12]:
import scipy.io as sio
import mat73
import numpy as np
import sys 
import os

In [13]:
def reconstruction_3D(dlt1, dlt2, dic2d):
    dlt1_mat = sio.loadmat(os.path.join(os.getcwd(), dlt1))
    dlt2_mat = sio.loadmat(os.path.join(os.getcwd(), dlt2))
    dltk = dlt1_mat['DLTstructCam']['DLTparams'][0][0] #the 11 dlt parameters for cam1 in matricx
    dltl = dlt2_mat['DLTstructCam']['DLTparams'][0][0] #the 11 dlt parameters for cam2 in matricx, and turn it in a float array

    dic2d = mat73.loadmat(os.path.join(os.getcwd(), dic2d) )
    dic2d_K = dic2d['DIC2DpairResults']['Points'][:5]
    dic2d_L = dic2d['DIC2DpairResults']['Points'][5:]

    p = []
    for j in range(len(dic2d_K)): 
        point = []
        dic2d_k = dic2d_K[j][0]
        dic2d_l = dic2d_L[j][0]
        for i in range(len(dic2d_k)):
            try:
                u = np.array([ [dic2d_k[i][0] - dltk[3]],
                                [dic2d_k[i][1] - dltk[7]],
                                [dic2d_l[i][0] - dltl[3]],
                                [dic2d_l[i][1] - dltl[7]] ])
                a = np.array([ [ dltk[0] - dltk[8]*dic2d_k[i][0], dltk[1] - dltk[9]*dic2d_k[i][0], dltk[2] - dltk[10]*dic2d_k[i][0] ],
                                [ dltk[4] - dltk[8]*dic2d_k[i][1], dltk[5] - dltk[9]*dic2d_k[i][1], dltk[6] - dltk[10]*dic2d_k[i][1] ],
                                [ dltl[0] - dltl[8]*dic2d_l[i][0], dltl[1] - dltl[9]*dic2d_l[i][0], dltl[2] - dltl[10]*dic2d_l[i][0] ],
                                [ dltl[4] - dltl[8]*dic2d_l[i][1], dltl[5] - dltl[9]*dic2d_l[i][1], dltl[6] - dltl[10]*dic2d_l[i][1] ], ])
                u = np.reshape(u, (4,1))
                a = np.reshape(a, (4,3))
                at = a.T
                tem = np.dot(at, a)
                tem1 = np.dot(np.linalg.inv(tem), a.T)
                p_3d = np.dot(tem1,u)
                point.append(p_3d)
            except:
                point.append(np.array([None,None,None]))
        p.append(point)
    return(p)

In [14]:
p = reconstruction_3D('DLTstruct_cam_201.mat', 'DLTstruct_cam_202.mat', 'DIC2DpairResults_C_201_C_202.mat')

In [15]:
len(p)

5

In [16]:
print('   X            Y            Z')
for i in p[0][:20]:
    print(i[0], i[1], i[2])
    print(' ')

   X            Y            Z
[1.03606647] [2.0493587] [248.61606988]
 
[1.01152849] [2.00633397] [247.68134727]
 
[1.0110167] [2.03033205] [246.66231736]
 
[1.03222457] [2.08106987] [245.6553864]
 
[1.05720863] [2.14498293] [244.6287584]
 
[1.06318622] [2.18633486] [243.58970968]
 
[1.03688722] [2.16951011] [242.58022776]
 
[1.03486627] [2.19421063] [241.55450174]
 
[1.02880183] [2.2118932] [240.53187003]
 
[0.98863017] [2.17340755] [239.52624621]
 
[0.97564001] [2.16022093] [238.55599412]
 
[0.92836019] [2.1151945] [237.54039544]
 
[0.92385788] [2.13257712] [236.52411344]
 
[0.86952216] [2.0690244] [235.528859]
 
[0.80079482] [1.97233086] [234.56332127]
 
[0.95487501] [2.26706825] [233.43274136]
 
[0.97522352] [2.31372194] [232.4341851]
 
[0.93649638] [2.28375852] [231.41218054]
 
[0.99854069] [2.43152185] [230.31594552]
 
[0.98802332] [2.45251076] [229.27103948]
 


In [17]:
def corrComb(dic2d):
    dic2d = mat73.loadmat(os.path.join(os.getcwd(),dic2d))
    k_corr = dic2d['DIC2DpairResults']['CorCoeffVec'][:5]
    l_corr = dic2d['DIC2DpairResults']['CorCoeffVec'][5:]
    corrComb = []
    for i in range(len(k_corr)):
        tem = []
        for j in range(len(k_corr[i][0])):
            k = k_corr[i][0]
            l = l_corr[i][0]
            cor = max(k[j], l[j])
            tem.append(cor)
        corrComb.append(tem)
    return(corrComb)

In [18]:
corr = corrComb('DIC2DpairResults_C_201_C_202.mat')
print(len(corr))

5


In [19]:
for i in corr[0][:10]:
     print(i)

1.6060218832135194
1.6105683120471774
1.608702157923486
1.5770957549161562
1.5422585639567723
1.5014034804986522
1.4967012763570755
1.4921421464399902
1.490811801104905
1.4759166586612957


In [20]:
def faceReturn(dic2d):
    dic2d = mat73.loadmat(os.path.join(os.getcwd(),dic2d))
    face = dic2d['DIC2DpairResults']['Faces']
    return(face)

In [21]:
face = faceReturn('DIC2DpairResults_C_201_C_202.mat')
print(len(face))

13368


In [22]:
for i in face[:10]:
    print(i)

[610. 338. 339.]
[4958. 4957. 4689.]
[2248. 1977. 1978.]
[2210. 2211. 2480.]
[2011. 2282. 2281.]
[1021. 1022. 1293.]
[4496. 4228. 4497.]
[2469. 2468. 2199.]
[  7. 279. 278.]
[323.  52. 324.]
